In [32]:
extraction_prompt = """You are a network graph maker with legal expertise who extracts terms and their relations from a given context. Your answer should help answer questions about one of the following themes:
You are provided with a context chunk (delimited by ```) Your task is to extract the ontology 
of terms mentioned in the given context. These terms should help answer questions on following themes:
    Document Name', 'Parties', 'Agreement Date', 'Effective Date', 'Expiration Date',
    Renewal Term', 'Notice Period To Terminate Renewal', 'Governing Law',
    Most Favored Nation', 'Competitive Restriction Exception', 'Non-Compete',
    Exclusivity', 'No-Solicit Of Customers', 'No-Solicit Of Employees',
    Non-Disparagement', 'Termination For Convenience', 'Rofr/Rofo/Rofn',
    Change Of Control', 'Anti-Assignment', 'Revenue/Profit Sharing', 'Price Restrictions',
    Minimum Commitment', 'Volume Restriction', 'Ip Ownership Assignment',
    Joint Ip Ownership', 'License Grant', 'Non-Transferable License',
    Affiliate License-Licensor', 'Affiliate License-Licensee',
    Unlimited/All-You-Can-Eat-License', 'Irrevocable Or Perpetual License',
    Source Code Escrow', 'Post-Termination Services', 'Audit Rights',
    Uncapped Liability', 'Cap On Liability', 'Liquidated Damages',
    Warranty Duration', 'Insurance', 'Covenant Not To Sue', 'Third Party Beneficiary'
Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.
    Terms may include object, entity, location, organization, person, 
    condition, acronym, documents, service, concept, etc.
    Terms should be as atomistic as possible
Thought 2: Think about how these terms can have one on one relation with other terms.
    Terms that are mentioned in the same sentence or the same paragraph are typically related to each other.
    Terms can be related to many other terms
Thought 3: Find out the relation between each such related pair of terms. 
Format your output as a list of knowledge triplets in the form of (subject, predicate, object) . Each element of the list contains a pair of terms
Don't write anything else except for extracted triplets and the relation between them, like the following: 
Text: Alice is Bob's mother.
Triplets:\n(Alice, is mother of, Bob)
Context: Philz is a coffee shop founded in Berkeley in 1982.
Triplets:
(Philz, is, coffee shop)
(Philz, founded in, Berkeley)
(Philz, founded in, 1982)
---------------------
context: {text}
Triplets:

"""

In [ ]:
import logging
import sys

from llama_index.llms.ollama import Ollama

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [5]:
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import Settings
from IPython.display import Markdown, display

In [14]:
documents = SimpleDirectoryReader(
    "data_test"
).load_data()

In [19]:
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model="llama3", request_timeout=120.0)
Settings.llm = llm
Settings.chunk_size = 1024
ollama_embedding = OllamaEmbedding(
    model_name="llama3",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = ollama_embedding

In [49]:
from llama_index.core import StorageContext
from llama_index.core import PromptTemplate

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)
extraction_template = PromptTemplate(extraction_prompt)

In [50]:
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=10,
    storage_context=storage_context,
    kg_triple_extract_template=extraction_template,
    include_embeddings=True,
    embed_model=ollama_embedding
)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://loc

In [110]:
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = index.as_retriever(
    include_text=True,
    retriever_mode="embedding",
    similarity_top_k=5,
    verbose=True,
    use_global_node_triplets=True,
    graph_store_query_depth=30)

query_engine = RetrieverQueryEngine.from_args(
    retriever,
    include_text=False,

)

response = query_engine.query(
    "What is the effective date?",
)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Extracted keywords: ['related', 'effective', 'keywords', 'words', 'question', 'extracted', 'date', 'relevant', "Here are the extracted keywords: 'KEYWORDS: effective", 'Here', 'find', 'These', 'two', 'help', 'information', 'KEYWORDS', "date'\n\nThese two words should help you find relevant information related to the question about the effective date."]
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: f21133d5-4fd5-4874-ac8e-1c6b6a41204e: in the case of the trademarks HERC RENTALS and HERCRENTALS Logo will be used ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 3c7694e9-1ac8-4ffd-8377-2f28d0cc26e6: This provision shall survive the expiration or earlier termination of this Ag...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 79b0576b-9aea-4e94-813f-6dc3ed33a609: 2.7 In the case of the HERC (HERC) Foreign ERB Trademar

In [111]:
response

Response(response='The effective date is not explicitly mentioned in the provided text. However, based on the context, it can be inferred that the agreement was signed or executed on July 7, 2016.', source_nodes=[NodeWithScore(node=TextNode(id_='f21133d5-4fd5-4874-ac8e-1c6b6a41204e', embedding=None, metadata={'file_path': '/Users/jalal/Documents/projects/KGRAG/data_test/HERTZGLOBALHOLDINGS,INC_07_07_2016-EX-10.4-INTELLECTUAL PROPERTY AGREEMENT.txt', 'file_name': 'HERTZGLOBALHOLDINGS,INC_07_07_2016-EX-10.4-INTELLECTUAL PROPERTY AGREEMENT.txt', 'file_type': 'text/plain', 'file_size': 30968, 'creation_date': '2024-05-20', 'last_modified_date': '2024-05-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6f00

In [112]:
display(Markdown(f"<b>{response}</b>"))


<b>The effective date is not explicitly mentioned in the provided text. However, based on the context, it can be inferred that the agreement was signed or executed on July 7, 2016.</b>

In [82]:
## create graph
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("example.html")

example.html
